# Calibration Tutorial - Fort Peck, MT - Unirrigated Flux Plot

## Step 3: Running the Calibrated Model

Now we evaluate whether calibration improved model performance by running in **forecast mode** with calibrated parameters.

This notebook:
1. Visualizes how parameters evolved during calibration
2. Runs the model with calibrated parameters
3. Compares calibrated vs uncalibrated performance against flux observations

In [ ]:
import os
import sys
import tempfile
import time

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score

root = os.path.abspath("../..")
sys.path.append(root)

from swimrs.container import SwimContainer
from swimrs.process.input import build_swim_input
from swimrs.process.loop_fast import run_daily_loop_fast
from swimrs.swim.config import ProjectConfig
from swimrs.viz.param_evolution import plot_parameter_histograms
from swimrs.viz.swim_timeseries import plot_swim_timeseries

%matplotlib inline

In [ ]:
project_ws = os.path.abspath(".")
data = os.path.join(project_ws, "data")
pestrun = os.path.join(data, "pestrun")
pest_dir = os.path.join(pestrun, "pest")

config_file = os.path.join(project_ws, "2_Fort_Peck.toml")

## 1. Visualize Parameter Evolution

Let's see how the parameters changed across optimization iterations. The histograms show the distribution of parameter values across ensemble realizations.

In [ ]:
initial_params = os.path.join(pestrun, "params.csv")

# Get all parameter files from optimization steps
steps = []
for i in range(10):  # Check up to 10 iterations
    step_file = os.path.join(pest_dir, f"2_Fort_Peck.{i}.par.csv")
    if os.path.exists(step_file):
        steps.append(step_file)

if steps:
    print(f"Found {len(steps)} optimization steps")

    fig_dir = os.path.join(project_ws, "figures", "parameter_hist")
    os.makedirs(fig_dir, exist_ok=True)

    # Plot histograms (set fig_out_dir=fig_dir to save PNGs)
    plot_parameter_histograms(initial_params, steps, fig_out_dir=None)
else:
    print("No parameter files found. Run notebook 02_calibration first.")

### Parameter Comparison Table

Compare initial parameter values with calibrated ensemble statistics:

In [ ]:
def show_parameter_comparison(initial_params_file, calibrated_params_file):
    """Display comparison of initial vs calibrated parameter values."""

    # Parameter descriptions
    param_descriptions = {
        "aw": "Available water capacity (mm)",
        "ks_alpha": "Evap stress coefficient",
        "kr_alpha": "Transpiration stress coefficient",
        "ndvi_k": "NDVI-Kcb slope",
        "ndvi_0": "NDVI at Kcb=0",
        "mad": "Mgmt allowed depletion",
        "swe_alpha": "SWE melt coefficient",
        "swe_beta": "SWE melt exponent",
    }

    # Known parameter names (to properly parse from column headers)
    known_params = [
        "aw",
        "ks_alpha",
        "kr_alpha",
        "ndvi_k",
        "ndvi_0",
        "mad",
        "swe_alpha",
        "swe_beta",
    ]

    # Read initial parameters
    initial_df = pd.read_csv(initial_params_file, index_col=0)

    # Read calibrated parameters (ensemble)
    cal_df = pd.read_csv(calibrated_params_file)

    # Extract parameter columns (skip 'real_name' column)
    param_cols = [c for c in cal_df.columns if c.startswith("pname:")]

    rows = []
    for col in param_cols:
        # Parse parameter name from column header like 'pname:p_ndvi_k_us-fpe_:0_ptype:cn_usecol:1_pstyle:m'
        parts = col.split(":")
        pname_part = parts[1]  # 'p_ndvi_k_us-fpe_'

        # Find which known parameter this column represents
        param_key = None
        for known in known_params:
            if pname_part.startswith(f"p_{known}_"):
                param_key = known
                break

        if param_key is None:
            continue

        # Get calibrated ensemble statistics
        cal_values = cal_df[col].values
        cal_mean = cal_values.mean()
        cal_std = cal_values.std()

        # Find matching initial parameter
        initial_val = None
        for idx in initial_df.index:
            if param_key in idx.lower():
                initial_val = initial_df.loc[idx, "value"]
                break

        if initial_val is not None:
            change = cal_mean - initial_val
            pct_change = (change / initial_val) * 100 if initial_val != 0 else 0

            rows.append(
                {
                    "Parameter": param_key,
                    "Initial": f"{initial_val:.3f}",
                    "Cal Mean": f"{cal_mean:.3f}",
                    "Cal Std": f"{cal_std:.3f}",
                    "Change": f"{change:+.3f}",
                    "% Change": f"{pct_change:+.1f}%",
                    "Description": param_descriptions.get(param_key, ""),
                }
            )

    result_df = pd.DataFrame(rows)

    print("=" * 95)
    print("PARAMETER COMPARISON: Initial vs Calibrated")
    print("=" * 95)
    print(result_df.to_string(index=False))
    print("=" * 95)

    return result_df


# Find the final calibrated parameter file
if steps:
    final_params = steps[-1]
    print(f"Using calibrated parameters from: {os.path.basename(final_params)}\n")
    param_comparison = show_parameter_comparison(initial_params, final_params)
else:
    # Check archive directory for calibrated parameters
    archive_dir = os.path.join(pest_dir, "archive")
    if os.path.exists(archive_dir):
        archive_files = sorted([f for f in os.listdir(archive_dir) if f.endswith(".par.csv")])
        if archive_files:
            final_params = os.path.join(archive_dir, archive_files[-1])
            print(f"Using archived calibrated parameters from: {archive_files[-1]}\n")
            param_comparison = show_parameter_comparison(initial_params, final_params)
        else:
            print("No calibrated parameter files found in archive.")

## 2. Run the Calibrated Model

To run with calibrated parameters:
1. Set `forecast=True` when reading config
2. Ensure `[forecast]` section in config points to the final `.par.csv` file

The config file should have:
```toml
[forecast]
forecast_parameters = "{pest_run_dir}/pest/2_Fort_Peck.3.par.csv"
```

In [ ]:
def run_fields(config, container, selected_feature, output_csv, forecast=False):
    """Run SWIM model using the process package and save output to CSV.

    Uses the modern process package workflow:
    1. Build SwimInput from container (with calibrated params if forecast=True)
    2. Run simulation with run_daily_loop_fast()
    3. Convert DailyOutput to DataFrame with time series
    4. Add ETf observations from container for comparison
    """
    start_time = time.time()

    # Create temporary HDF5 for SwimInput
    with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
        temp_h5_path = tmp.name

    # Handle calibrated parameters for forecast mode
    calibrated_params_path = None
    if forecast:
        # Read forecast parameters and convert to JSON
        config.read_forecast_parameters()
        if hasattr(config, "forecast_parameters") and config.forecast_parameters is not None:
            calibrated_params_path = _convert_forecast_params_to_json(
                config.forecast_parameters, os.path.dirname(output_csv)
            )

    try:
        # Build SwimInput from container
        swim_input = build_swim_input(
            container,
            output_h5=temp_h5_path,
            calibrated_params_path=calibrated_params_path,
            runoff_process=getattr(config, "runoff_process", "cn"),
            etf_model=getattr(config, "etf_target_model", "ptjpl"),
            met_source=getattr(config, "met_source", "gridmet"),
            fields=[selected_feature],
        )

        # Run simulation (uses fast JIT-compiled loop)
        output, final_state = run_daily_loop_fast(swim_input)

        # Get time series data
        n_days = swim_input.n_days
        dates = pd.date_range(swim_input.start_date, periods=n_days, freq="D")

        # Get input time series for DataFrame
        etr = swim_input.get_time_series("etr")
        prcp = swim_input.get_time_series("prcp")
        tmin = swim_input.get_time_series("tmin")
        tmax = swim_input.get_time_series("tmax")
        ndvi = swim_input.get_time_series("ndvi")

        # Build DataFrame (field index 0 since we're doing single field)
        i = 0
        df_data = {
            # Model outputs
            "et_act": output.eta[:, i],
            "etref": etr[:, i],
            "kc_act": output.etf[:, i],
            "kc_bas": output.kcb[:, i],
            "ks": output.ks[:, i],
            "ke": output.ke[:, i],
            "melt": output.melt[:, i],
            "rain": output.rain[:, i],
            "depl_root": output.depl_root[:, i],
            "dperc": output.dperc[:, i],
            "runoff": output.runoff[:, i],
            "swe": output.swe[:, i],
            "irrigation": output.irr_sim[:, i],
            "gw_sim": output.gw_sim[:, i],
            # Input time series
            "ppt": prcp[:, i],
            "tmin": tmin[:, i],
            "tmax": tmax[:, i],
            "tavg": (tmin[:, i] + tmax[:, i]) / 2.0,
            "ndvi": ndvi[:, i],
        }

        # Calculate derived columns
        df_data["soil_water"] = swim_input.properties.awc[i] - output.depl_root[:, i]

        df = pd.DataFrame(df_data, index=dates)

        swim_input.close()

        # Load ETf observations from container for comparison
        # These are the remote sensing observations (PT-JPL in this example)
        etf_model = getattr(config, "etf_target_model", "ptjpl")
        for mask in ["inv_irr", "irr"]:
            etf_path = f"remote_sensing/etf/landsat/{etf_model}/{mask}"
            try:
                etf_df = container.query.dataframe(etf_path, fields=[selected_feature])
                # Align with simulation dates
                etf_series = etf_df[selected_feature].reindex(dates)
                df[f"etf_{mask}"] = etf_series.values
            except Exception as e:
                print(f"Warning: Could not load ETf for {mask}: {e}")
                df[f"etf_{mask}"] = np.nan

    finally:
        # Clean up temp files
        if os.path.exists(temp_h5_path):
            os.remove(temp_h5_path)
        if calibrated_params_path and os.path.exists(calibrated_params_path):
            os.remove(calibrated_params_path)

    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds\n")

    df.to_csv(output_csv)

    return df


def _convert_forecast_params_to_json(forecast_params, out_dir: str) -> str:
    """Convert forecast_parameters Series to JSON format for build_swim_input.

    The forecast_parameters Series has index like 'kc_max_FID1', 'ndvi_k_FID1', etc.
    We convert to: {FID1: {kc_max: val, ndvi_k: val, ...}, ...}
    """
    import json
    import tempfile

    params_by_fid = {}
    for param_name in forecast_params.index:
        # Parse param name: expect format like 'kc_max_FID1' or 'ndvi_k_FID1'
        parts = param_name.rsplit("_", 1)
        if len(parts) == 2:
            base_param, fid = parts
            if fid not in params_by_fid:
                params_by_fid[fid] = {}
            params_by_fid[fid][base_param] = float(forecast_params[param_name])

    # Write to temp JSON file
    fd, json_path = tempfile.mkstemp(suffix=".json", prefix="calib_params_", dir=out_dir)
    os.close(fd)
    with open(json_path, "w") as f:
        json.dump(params_by_fid, f)

    return json_path

In [ ]:
selected_feature = "US-FPe"
out_csv = os.path.join(project_ws, f"combined_output_{selected_feature}_calibrated.csv")

# Open container for model run
container_path = os.path.join(data, "2_Fort_Peck.swim")
container = SwimContainer.open(container_path, mode="r")

# Load config with forecast=True to get calibrated parameter path
config = ProjectConfig()
config.read_config(config_file, project_ws, forecast=True)

try:
    # Run with forecast=True to use calibrated parameters
    df = run_fields(
        config, container, selected_feature=selected_feature, output_csv=out_csv, forecast=True
    )
finally:
    container.close()

## 3. Visualize Calibrated Output

In [ ]:
ydf = df.loc["2004-01-01":"2004-12-31"]
print(f"Total irrigation: {ydf.irrigation.sum():.1f} mm")
print(f"Total ET: {ydf.et_act.sum():.1f} mm")
print(f"Total precipitation: {ydf.ppt.sum():.1f} mm")

plot_swim_timeseries(
    ydf,
    ["et_act", "etref", "rain", "melt", "irrigation"],
    start="2004-01-01",
    end="2004-12-31",
    png_dir="et_calibrated.png",
)

## 4. Compare with Flux Tower Observations

We show two comparisons:
- **Full time series**: SWIM (daily) vs PT-JPL (interpolated between Landsat dates) on all flux tower days
- **Capture dates only**: Both methods compared only on Landsat overpass dates

In [ ]:
def compare_et_estimates(combined_output_path, flux_data_path, irr=False, label=""):
    """Compare model ET and PT-JPL ET against flux tower observations.

    Returns two comparison DataFrames:
    1. Capture dates only: Both methods on Landsat overpass dates only
    2. Full time series: SWIM daily, PT-JPL interpolated, on all flux tower days
    """
    flux_data = pd.read_csv(flux_data_path, index_col="date", parse_dates=True)
    flux_et = flux_data["ET_corr"]  # Energy-balance-corrected ET from flux tower (mm/day)

    output = pd.read_csv(combined_output_path, index_col=0)
    output.index = pd.to_datetime(output.index)

    etf_col = "etf_irr" if irr else "etf_inv_irr"
    ptjpl_et_sparse = output[etf_col] * output["etref"]
    ptjpl_et_interp = ptjpl_et_sparse.interpolate(method="linear")
    n_ptjpl_obs = ptjpl_et_sparse.notna().sum()

    # CAPTURE DATES ONLY comparison (PT-JPL sparse)
    capture_df = pd.DataFrame(
        {"swim_et": output["et_act"], "ptjpl_et": ptjpl_et_sparse, "flux_et": flux_et}
    ).dropna()

    # FULL TIME SERIES comparison (PT-JPL interpolated)
    full_df = pd.DataFrame(
        {"swim_et": output["et_act"], "ptjpl_et": ptjpl_et_interp, "flux_et": flux_et}
    ).dropna()

    def calc_metrics(df, col1, col2):
        r, _ = stats.pearsonr(df[col1], df[col2])
        r2 = r2_score(df[col1], df[col2])
        rmse = np.sqrt(mean_squared_error(df[col1], df[col2]))
        bias = (df[col2] - df[col1]).mean()
        return r2, r, rmse, bias

    # Capture dates metrics
    r2_swim_cap, r_swim_cap, rmse_swim_cap, bias_swim_cap = calc_metrics(
        capture_df, "flux_et", "swim_et"
    )
    r2_ptjpl_cap, r_ptjpl_cap, rmse_ptjpl_cap, bias_ptjpl_cap = calc_metrics(
        capture_df, "flux_et", "ptjpl_et"
    )

    # Full time series metrics
    r2_swim_full, r_swim_full, rmse_swim_full, bias_swim_full = calc_metrics(
        full_df, "flux_et", "swim_et"
    )
    r2_ptjpl_full, r_ptjpl_full, rmse_ptjpl_full, bias_ptjpl_full = calc_metrics(
        full_df, "flux_et", "ptjpl_et"
    )

    if label:
        print(f"\n{label}")
        print("=" * 70)

    print(f"CAPTURE DATES ONLY ({len(capture_df)} Landsat overpass dates)")
    print("-" * 70)
    print(f"{'Metric':<12} {'SWIM ET':>12} {'PT-JPL ET':>12}")
    print("-" * 38)
    print(f"{'R²':<12} {r2_swim_cap:>12.3f} {r2_ptjpl_cap:>12.3f}")
    print(f"{'Pearson r':<12} {r_swim_cap:>12.3f} {r_ptjpl_cap:>12.3f}")
    print(f"{'Bias (mm)':<12} {bias_swim_cap:>12.3f} {bias_ptjpl_cap:>12.3f}")
    print(f"{'RMSE (mm)':<12} {rmse_swim_cap:>12.3f} {rmse_ptjpl_cap:>12.3f}")

    print()
    print(f"FULL TIME SERIES ({len(full_df)} days, PT-JPL interpolated from {n_ptjpl_obs} obs)")
    print("-" * 70)
    print(f"{'Metric':<12} {'SWIM ET':>12} {'PT-JPL ET':>12}")
    print("-" * 38)
    print(f"{'R²':<12} {r2_swim_full:>12.3f} {r2_ptjpl_full:>12.3f}")
    print(f"{'Pearson r':<12} {r_swim_full:>12.3f} {r_ptjpl_full:>12.3f}")
    print(f"{'Bias (mm)':<12} {bias_swim_full:>12.3f} {bias_ptjpl_full:>12.3f}")
    print(f"{'RMSE (mm)':<12} {rmse_swim_full:>12.3f} {rmse_ptjpl_full:>12.3f}")

    # Return metrics for summary comparison
    return (
        full_df,
        capture_df,
        {
            "capture": {
                "r2": r2_swim_cap,
                "r": r_swim_cap,
                "rmse": rmse_swim_cap,
                "bias": bias_swim_cap,
            },
            "full": {
                "r2": r2_swim_full,
                "r": r_swim_full,
                "rmse": rmse_swim_full,
                "bias": bias_swim_full,
            },
        },
    )

In [ ]:
flux_data = os.path.join(data, "US-FPe_daily_data.csv")
full_df, capture_df, metrics_cal = compare_et_estimates(out_csv, flux_data, label="CALIBRATED")

### Compare with Uncalibrated Results

In [ ]:
uncal_csv = os.path.join(project_ws, f"combined_output_{selected_feature}_uncalibrated.csv")

if os.path.exists(uncal_csv):
    _, _, metrics_uncal = compare_et_estimates(uncal_csv, flux_data, label="UNCALIBRATED")

    print("\n" + "=" * 70)
    print("CALIBRATION IMPROVEMENT SUMMARY (SWIM vs Flux)")
    print("=" * 70)

    print("\nCapture Dates Only:")
    print(
        f"  R²:           {metrics_uncal['capture']['r2']:.3f} -> {metrics_cal['capture']['r2']:.3f}  ({metrics_cal['capture']['r2'] - metrics_uncal['capture']['r2']:+.3f})"
    )
    print(
        f"  Pearson r:    {metrics_uncal['capture']['r']:.3f} -> {metrics_cal['capture']['r']:.3f}  ({metrics_cal['capture']['r'] - metrics_uncal['capture']['r']:+.3f})"
    )
    print(
        f"  Bias (mm):    {metrics_uncal['capture']['bias']:+.3f} -> {metrics_cal['capture']['bias']:+.3f}"
    )
    print(
        f"  RMSE (mm):    {metrics_uncal['capture']['rmse']:.3f} -> {metrics_cal['capture']['rmse']:.3f}  ({metrics_cal['capture']['rmse'] - metrics_uncal['capture']['rmse']:+.3f})"
    )

    print("\nFull Time Series:")
    print(
        f"  R²:           {metrics_uncal['full']['r2']:.3f} -> {metrics_cal['full']['r2']:.3f}  ({metrics_cal['full']['r2'] - metrics_uncal['full']['r2']:+.3f})"
    )
    print(
        f"  Pearson r:    {metrics_uncal['full']['r']:.3f} -> {metrics_cal['full']['r']:.3f}  ({metrics_cal['full']['r'] - metrics_uncal['full']['r']:+.3f})"
    )
    print(
        f"  Bias (mm):    {metrics_uncal['full']['bias']:+.3f} -> {metrics_cal['full']['bias']:+.3f}"
    )
    print(
        f"  RMSE (mm):    {metrics_uncal['full']['rmse']:.3f} -> {metrics_cal['full']['rmse']:.3f}  ({metrics_cal['full']['rmse'] - metrics_uncal['full']['rmse']:+.3f})"
    )
else:
    print("Uncalibrated output not found. Run notebook 01 first.")

In [ ]:
import matplotlib.pyplot as plt

# Create scatter plots for both comparisons
fig, axes = plt.subplots(2, 2, figsize=(12, 10))


# Helper function to calculate metrics
def calc_metrics(df, col1, col2):
    r, _ = stats.pearsonr(df[col1], df[col2])
    r2 = r2_score(df[col1], df[col2])
    rmse = np.sqrt(mean_squared_error(df[col1], df[col2]))
    return r2, r, rmse


# Determine axis limits
max_et = max(full_df["flux_et"].max(), full_df["swim_et"].max(), full_df["ptjpl_et"].max()) * 1.1

# TOP ROW: Capture dates only
r2_swim_cap, r_swim_cap, rmse_swim_cap = calc_metrics(capture_df, "flux_et", "swim_et")
r2_ptjpl_cap, r_ptjpl_cap, rmse_ptjpl_cap = calc_metrics(capture_df, "flux_et", "ptjpl_et")

ax = axes[0, 0]
ax.scatter(capture_df["flux_et"], capture_df["swim_et"], alpha=0.5, s=15)
ax.plot([0, max_et], [0, max_et], "r--", label="1:1 line")
ax.set_xlabel("Flux ET (mm/day)")
ax.set_ylabel("SWIM ET (mm/day)")
ax.set_title(
    f"SWIM vs Flux - Capture Dates (n={len(capture_df)})\n"
    f"R² = {r2_swim_cap:.3f}, r = {r_swim_cap:.3f}, RMSE = {rmse_swim_cap:.2f} mm"
)
ax.legend()
ax.set_xlim(0, max_et)
ax.set_ylim(0, max_et)

ax = axes[0, 1]
ax.scatter(capture_df["flux_et"], capture_df["ptjpl_et"], alpha=0.5, s=15)
ax.plot([0, max_et], [0, max_et], "r--", label="1:1 line")
ax.set_xlabel("Flux ET (mm/day)")
ax.set_ylabel("PT-JPL ET (mm/day)")
ax.set_title(
    f"PT-JPL vs Flux - Capture Dates (n={len(capture_df)})\n"
    f"R² = {r2_ptjpl_cap:.3f}, r = {r_ptjpl_cap:.3f}, RMSE = {rmse_ptjpl_cap:.2f} mm"
)
ax.legend()
ax.set_xlim(0, max_et)
ax.set_ylim(0, max_et)

# BOTTOM ROW: Full time series comparison
r2_swim, r_swim, rmse_swim = calc_metrics(full_df, "flux_et", "swim_et")
r2_ptjpl, r_ptjpl, rmse_ptjpl = calc_metrics(full_df, "flux_et", "ptjpl_et")

ax = axes[1, 0]
ax.scatter(full_df["flux_et"], full_df["swim_et"], alpha=0.3, s=10)
ax.plot([0, max_et], [0, max_et], "r--", label="1:1 line")
ax.set_xlabel("Flux ET (mm/day)")
ax.set_ylabel("SWIM ET (mm/day)")
ax.set_title(
    f"SWIM vs Flux - Full Series (n={len(full_df)})\n"
    f"R² = {r2_swim:.3f}, r = {r_swim:.3f}, RMSE = {rmse_swim:.2f} mm"
)
ax.legend()
ax.set_xlim(0, max_et)
ax.set_ylim(0, max_et)

ax = axes[1, 1]
ax.scatter(full_df["flux_et"], full_df["ptjpl_et"], alpha=0.3, s=10)
ax.plot([0, max_et], [0, max_et], "r--", label="1:1 line")
ax.set_xlabel("Flux ET (mm/day)")
ax.set_ylabel("PT-JPL ET (mm/day)")
ax.set_title(
    f"PT-JPL vs Flux - Full Series, interpolated (n={len(full_df)})\n"
    f"R² = {r2_ptjpl:.3f}, r = {r_ptjpl:.3f}, RMSE = {rmse_ptjpl:.2f} mm"
)
ax.legend()
ax.set_xlim(0, max_et)
ax.set_ylim(0, max_et)

plt.tight_layout()
plt.savefig("comparison_scatter_calibrated.png", dpi=150)
plt.show()

## 5. Mass Balance Check

Verify water conservation across the full simulation period. The water balance equation:

**Starting Water + Inputs - Outputs = Ending Water**

Where:
- **Starting Water** = Initial Soil Water + Initial SWE
- **Inputs** = Precipitation + Irrigation  
- **Outputs** = ET + Deep Percolation + Runoff
- **Ending Water** = Final Soil Water + Final SWE

In [ ]:
def compute_mass_balance(df):
    """Compute full-run mass balance and display as a table.

    Water Balance: Starting Water + Inputs - Outputs = Ending Water

    Components:
    - Starting Water: Initial soil water + Initial SWE
    - Inputs: Total precipitation + Total irrigation
    - Outputs: Total ET + Total deep percolation + Total runoff
    - Ending Water: Final soil water + Final SWE
    """
    # Starting conditions (first day values)
    start_soil_water = df["soil_water"].iloc[0]
    start_swe = df["swe"].iloc[0]
    start_water = start_soil_water + start_swe

    # Ending conditions (last day values)
    end_soil_water = df["soil_water"].iloc[-1]
    end_swe = df["swe"].iloc[-1]
    end_water = end_soil_water + end_swe

    # Inputs (sums over full period)
    total_precip = df["ppt"].sum()
    total_irrigation = df["irrigation"].sum()
    total_inputs = total_precip + total_irrigation

    # Outputs (sums over full period)
    total_et = df["et_act"].sum()
    total_dperc = df["dperc"].sum()
    total_runoff = df["runoff"].sum()
    total_outputs = total_et + total_dperc + total_runoff

    # Mass balance check
    # Expected: Start + Inputs - Outputs = End
    # Residual: (Start + Inputs - Outputs) - End = 0 if balanced
    expected_end = start_water + total_inputs - total_outputs
    residual = expected_end - end_water
    pct_error = (residual / total_inputs) * 100 if total_inputs > 0 else 0

    # Create summary table
    print("=" * 70)
    print("MASS BALANCE SUMMARY")
    print(
        f"Period: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')} ({len(df)} days)"
    )
    print("=" * 70)

    print("\n--- STARTING WATER ---")
    print(f"  Initial Soil Water:     {start_soil_water:>10.2f} mm")
    print(f"  Initial SWE:            {start_swe:>10.2f} mm")
    print(f"  TOTAL STARTING:         {start_water:>10.2f} mm")

    print("\n--- INPUTS (+) ---")
    print(f"  Precipitation:          {total_precip:>10.2f} mm")
    print(f"  Irrigation:             {total_irrigation:>10.2f} mm")
    print(f"  TOTAL INPUTS:           {total_inputs:>10.2f} mm")

    print("\n--- OUTPUTS (-) ---")
    print(f"  Evapotranspiration:     {total_et:>10.2f} mm")
    print(f"  Deep Percolation:       {total_dperc:>10.2f} mm")
    print(f"  Runoff:                 {total_runoff:>10.2f} mm")
    print(f"  TOTAL OUTPUTS:          {total_outputs:>10.2f} mm")

    print("\n--- ENDING WATER ---")
    print(f"  Final Soil Water:       {end_soil_water:>10.2f} mm")
    print(f"  Final SWE:              {end_swe:>10.2f} mm")
    print(f"  TOTAL ENDING:           {end_water:>10.2f} mm")

    print("\n--- BALANCE CHECK ---")
    print(f"  Expected End (Start + In - Out):  {expected_end:>10.2f} mm")
    print(f"  Actual End:                       {end_water:>10.2f} mm")
    print(f"  Residual (Error):                 {residual:>10.2f} mm")
    print(f"  Error as % of Inputs:             {pct_error:>10.4f} %")
    print("=" * 70)

    if abs(pct_error) < 0.01:
        print("MASS BALANCE: CLOSED (error < 0.01%)")
    elif abs(pct_error) < 1.0:
        print("MASS BALANCE: ACCEPTABLE (error < 1%)")
    else:
        print(f"MASS BALANCE: WARNING - {abs(pct_error):.2f}% error")

    return {
        "start_soil_water": start_soil_water,
        "start_swe": start_swe,
        "end_soil_water": end_soil_water,
        "end_swe": end_swe,
        "precip": total_precip,
        "irrigation": total_irrigation,
        "et": total_et,
        "dperc": total_dperc,
        "runoff": total_runoff,
        "residual": residual,
        "pct_error": pct_error,
    }


# Run mass balance check
balance = compute_mass_balance(df)

## Conclusion ##

Calibration improved model performance:

- **Correlation improved** from uncalibrated to calibrated, while the metric improvement is modest, the tightening of the values around the 1:1 line is clear in comparison
- **SWIM outperforms PT-JPL** in both correlation and bias when comparing actual ET (mm/day)
- **PT-JPL performance is worse** during days without a satellite overpass
- **SWIM suffers no performance penalty** on between-capture dates

### Why does this work?

**The key insight is that we can mine the deep remote sensing-based ET record, but rather than driving the model with remote sensing ET directly, we drive the calibration with it.**

The model has access to:
1. Daily meteorological data (not just satellite overpass days)
2. Physically-based soil water balance constraints (if the soil is dry, ET slows)
3. Flexibility to tune parameters using the remote sensing record

This combination gives SWIM a more grounded perspective on daily fluxes than remote sensing alone, resulting in better ET estimates. The fact that SWIM is tracking the available soil moisture on a daily basis allows it to respond to conditions through time, rather than simply interpolating the underlying ET driver (ETf) between capture dates. If we simply drove a naive soil water balance with remote sensing data, we'd fall victim to the same degredation in performance between satellite overpasses. When we use the capture date information to refine an already-good hydrological model, we enable it to find a parameterization that can match or exceed capture date remote sensing accuracy and also make good estimates on days with no remote sensing data.

Next, we will use a more sophisticaed approach in an irrigated setting, a combination of the open-source OpenET ensemble members.
